# Step 2: Train a spacy model with NE-Samples extracted from TEIs and evaluate the results

The notebook used here comes from the workshop "Information Extraction aus frühneuhochdeutschen Texten" (https://informationsmodellierung.uni-graz.at/de/neuigkeiten/detail/article/workshop-information-extraction-aus-fruehneuhochdeutschen-texten/). It was modified and adapted for this project.

In [19]:
from spacytei.train import batch_train
from spacytei.data_prep import csv_to_traindata, clean_train_data


## 1) Load a csv with training data
an csv was created with `step1_preprocessing_data_for_NER.ipynb`

In [20]:
TRAIN_DATA = csv_to_traindata('output_csv/samples_out_sents.csv')

FileNotFoundError: [Errno 2] File b'output_csv/samples_out_sents.csv' does not exist: b'output_csv/samples_out_sents.csv'

In [3]:
len(TRAIN_DATA)

2658

## 2) Clean train data

remove all empty examples, use only samles with text lenght greater 15

In [4]:
TRAIN_DATA = clean_train_data(TRAIN_DATA, min_ents=1, min_text_len=15, lang=[])
len(TRAIN_DATA)

1128

In [5]:
print(TRAIN_DATA)

[['Daß man in Betrachtung dessen billig mit dem Könige David sich verwundern und ausrufen mag:', {'entities': [(52, 57, 'PER')]}], ['Gleich wie aber unter den vier Teilen der Welt Asien jederzeit für das herrlichste und fürnehmste ist geschätzt worden:', {'entities': [(47, 52, 'LOC')]}], ['Also auch in Asien unter den Orientalischen Ländern das Persische Königreich und solches vieler Ursachen sonderlich aber der herrlich gewesenen Monarchie halber das auch ihr Gedächtnis bei den Liebhabern der Historien noch heute das Land kommentieret, und dessen jetzigen Zustand zuwissen eine sondere Begierde erwecket.', {'entities': [(13, 18, 'LOC')]}], ['Reich der Perser von seinem Alten in den Historien beschriebenen Zustande so gar abgegangen das man in Respekt dessen wohl Persien in Persien suchen und nicht finden sollte.', {'entities': [(123, 130, 'LOC'), (134, 141, 'LOC')]}], ['Wie durch occasion der neulicher Zeit aus Holstein an den König in Persien', {'entities': [(67, 74, 'LOC')]}], ['Scha

In [6]:
import pandas as pd

In [7]:
df = pd.DataFrame(TRAIN_DATA)
df.info()
df.to_csv('output_csv/samples_out_sents.csv', index=False) #save csv file to output_csv/samples_out_sents.csv

# Watch out! Do not overwrite "samples_out_sents_clean.csv" or the training won't work! 
# samples_out_sents_clean.csv was improved by hand, because the automatic function does not work properly

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1128 entries, 0 to 1127
Data columns (total 2 columns):
0    1128 non-null object
1    1128 non-null object
dtypes: object(2)
memory usage: 17.7+ KB


In [18]:
TRAIN_DATA = csv_to_traindata('output_csv/samples_out_sents_clean.csv') 

## 3) Train the model

Unfortunately I found out that the training data was not converted correctly into the spaCy format, partly some entities were not assigned correctly, these errors had to be fixed by hand in the training data. Some entities in the training data are still not aligned correctly, but they will be ignored during the training process.

At the end of the training the F-Score of the new model is obtained. 

In [17]:
batch_train(model='de_core_news_sm', train_data=TRAIN_DATA, output_dir='custom_model')

Loaded model 'de_core_news_sm'
730 train vs 314 test samples
Iteration Number: 0


C:\Users\jacky\Anaconda3\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "welche an statt des Zaums mit sehr großen Silbern ..." with entities "[(164, 166, 'PER')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)
C:\Users\jacky\Anaconda3\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Nach diesem sprach er mit vielen Worten den Segen ..." with entities "[(66, 68, 'PER'), (131, 133, 'PER'), (150, 154, 'P...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)
C:\Users\jacky\Anaconda3\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Da denn 

C:\Users\jacky\Anaconda3\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Es nennen ihn auch die Großfürstl: Dolmetscher und..." with entities "[(16, 18, 'PER'), (42, 44, 'PER'), (61, 67, 'LOC')...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)
C:\Users\jacky\Anaconda3\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "die Designation der Fürstl: Geschenke welche auch ..." with entities "[(20, 26, 'PER'), (50, 59, 'PER'), (115, 116, 'PER...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)
C:\Users\jacky\Anaconda3\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could no

C:\Users\jacky\Anaconda3\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Die Nachtwache in den Häusern." with entities "[(0, 2, 'PER')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)
C:\Users\jacky\Anaconda3\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Zentner schwer und zur Zeit des Großfürsten Boris ..." with entities "[(32, 36, 'PER'), (44, 49, 'PER'), (50, 57, 'PER')...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)
C:\Users\jacky\Anaconda3\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Die Gesandten taten vor die empfang

C:\Users\jacky\Anaconda3\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Dahin auch ihre gewöhnliche arten zureden gehen Al..." with entities "[(8, 10, 'PER'), (24, 26, 'PER'), (70, 72, 'PER'),...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)
C:\Users\jacky\Anaconda3\lib\site-packages\spacytei\train.py:16: UserWarning: [W030] Some entities could not be aligned in the text "Die Dorpatische Academia, wenn sie angefangen." with entities "[(0, 2, 'PER'), (4, 6, 'PER')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc_gold_text, entities=x[1]['entities'])
C:\Users\jacky\Anaconda3\lib\site-packages\spacytei\train.py:16: UserWarning: [W030] Some entities could not b

Losses {'ner': 2594.4479683753434}
Duration: 0:00:09.775593


C:\Users\jacky\Anaconda3\lib\site-packages\spacytei\train.py:16: UserWarning: [W030] Some entities could not be aligned in the text "Und als hernach die Ritterschaft bei den Russische..." with entities "[(109, 117, 'LOC'), (135, 147, 'PER'), (202, 207, ...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc_gold_text, entities=x[1]['entities'])
C:\Users\jacky\Anaconda3\lib\site-packages\spacytei\train.py:16: UserWarning: [W030] Some entities could not be aligned in the text "Crusius und dich Otto Brüggeman zu empfangen und i..." with entities "[(0, 7, 'PER'), (14, 16, 'PER'), (17, 21, 'PER')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc_gold_text, entities=x[1]['entities'])
C:\Users\jacky\Anaconda3\lib\site-packages\spacytei\trai

C:\Users\jacky\Anaconda3\lib\site-packages\spacytei\train.py:16: UserWarning: [W030] Some entities could not be aligned in the text "Wenn auch der Großfürst mit jemand redet tituliere..." with entities "[(7, 9, 'PER'), (61, 63, 'PER')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc_gold_text, entities=x[1]['entities'])
C:\Users\jacky\Anaconda3\lib\site-packages\spacytei\train.py:16: UserWarning: [W030] Some entities could not be aligned in the text "Wir hielten uns daselbst in die sechs Wochen auf M..." with entities "[(49, 51, 'PER'), (80, 82, 'PER'), (83, 94, 'PER')...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc_gold_text, entities=x[1]['entities'])
C:\Users\jacky\Anaconda3\lib\site-packages\spacytei\train.py:16: UserWarn

C:\Users\jacky\Anaconda3\lib\site-packages\spacytei\train.py:16: UserWarning: [W030] Some entities could not be aligned in the text "Wer kann wieder Gott und Groß- Newgart aber es hat..." with entities "[(12, 14, 'PER'), (25, 29, 'PER'), (31, 38, 'PER')...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc_gold_text, entities=x[1]['entities'])
C:\Users\jacky\Anaconda3\lib\site-packages\spacytei\train.py:16: UserWarning: [W030] Some entities could not be aligned in the text "Sie begnadiget hätte mit ihren Hoffe Junkern vor i..." with entities "[(37, 39, 'PER')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc_gold_text, entities=x[1]['entities'])
C:\Users\jacky\Anaconda3\lib\site-packages\spacytei\train.py:16: UserWarning: [W030] Som

p: 63.48122866894198; f: 49.86595174262734; r: 41.05960264900662
Saved model to: custom_model
######################
Iteration Number: 1
Losses {'ner': 1992.4132588363864}
Duration: 0:00:09.458177
p: 63.61256544502618; f: 58.20359281437126; r: 53.64238410596026
Saved model to: custom_model
######################
Iteration Number: 2
Losses {'ner': 1776.693790466211}
Duration: 0:00:09.427088
p: 66.75392670157068; f: 61.07784431137725; r: 56.29139072847682
Saved model to: custom_model
######################
Iteration Number: 3
Losses {'ner': 1647.1467676544223}
Duration: 0:00:09.922459
p: 63.95061728395062; f: 60.37296037296037; r: 57.17439293598234
######################
Iteration Number: 4
Losses {'ner': 1610.2565933743472}
Duration: 0:00:09.487855
p: 66.66666666666666; f: 62.19081272084806; r: 58.27814569536424
Saved model to: custom_model
######################
Iteration Number: 5
Losses {'ner': 1473.2798924683584}
Duration: 0:00:09.585083
p: 67.32673267326733; f: 63.47724620770128; r

## 4) Evaluation of the German standard models

To get a comparison to the standard models, the German standard models of spaCy could be evaluated here. 

In [ ]:
import spacy
from spacy.gold import GoldParse
from spacy.scorer import Scorer

def evaluate(ner_model, examples):
    scorer = Scorer()
    for x in examples:
        doc_gold_text = ner_model.make_doc(x[0])
        gold = GoldParse(doc_gold_text, entities=x[1]['entities'])
        pred_value = ner_model(x[0])
        scorer.score(pred_value, gold)
    return scorer.scores

# example run

examples = TRAIN_DATA

# evaluate standard models
ner_model = spacy.load('de_core_news_sm') # you could put in here an other model like "de_core_news_md" to check the F-Score
results = evaluate(ner_model, examples)

results